In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql import functions as F

In [0]:
# retrive the file address
# dbutils.fs.ls('dbfs:/FileStore/shared_uploads/cfu11@sheffield.ac.uk/')

In [0]:
#load the first dataset (jds_50k)
# csv_jds = spark.read.format("csv").option("header", "false").load("dbfs:/FileStore/shared_uploads/cfu11@sheffield.ac.uk/skill2vec_50K.csv")
# use this code to load the data for submition:
csv_jds = spark.read.format("csv").option("header", "false").load("dbfs:/FileStore/tables/skill2vec_50K.csv.gz")

In [0]:
csv_jds.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------------+-------------+----------+-----------+-----------+------------------+--------------------+------------+--------------+-----------+-----------------+-------------+-----------+--------------------+-----------+------+------------+----------------+-------------------+--------------------+------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----

In [0]:
len(csv_jds.columns)

Out[35]: 961

In [0]:
# load the second dataset (txt file)
# txt_ts = spark.read.text("dbfs:/FileStore/shared_uploads/cfu11@sheffield.ac.uk/Technology_Skills.txt")
# use this code to load the data for submition:
txt_ts = spark.read.text("dbfs:/FileStore/tables/Technology_Skills.txt")

In [0]:
# txt_ts.show(truncate = False)

In [0]:
## import the test data
# csv_jds_1k = spark.read.format("csv").option("header", "false").load("dbfs:/FileStore/shared_uploads/cfu11@sheffield.ac.uk/skill2vec_1K.csv")
# csv_jds_10k = spark.read.format("csv").option("header", "false").load("dbfs:/FileStore/shared_uploads/cfu11@sheffield.ac.uk/skill2vec_10K.csv")
## copy the test data
# jds = csv_jds_10k

In [0]:
# question 1
csv_jds.count()

Out[4]: 50000

In [0]:
# question 1 check
csv_jds.distinct().count()

Out[5]: 50000

In [0]:
# Question 2

In [0]:
# csv pre-processing
# Create a dataframe which contains arrays in a single column
jds_array = csv_jds.select(F.array(csv_jds.columns).alias("jds"))
jds_array.show(10)

+--------------------+
|                 jds|
+--------------------+
|[125720, HR Execu...|
|[112708, Special ...|
|[115226, consulti...|
|[19805, diploma, ...|
|[80208, Compensat...|
|[64086, Storage A...|
|[48468, HR Operat...|
|[122729, Simulink...|
|[36721, developme...|
|[9342, software_d...|
+--------------------+
only showing top 10 rows



In [0]:
# add size (test)
jds_size = jds_array.withColumn("num_jds", F.size("jds"))
jds_size.show(10)

+--------------------+-------+
|                 jds|num_jds|
+--------------------+-------+
|[125720, HR Execu...|    961|
|[112708, Special ...|    961|
|[115226, consulti...|    961|
|[19805, diploma, ...|    961|
|[80208, Compensat...|    961|
|[64086, Storage A...|    961|
|[48468, HR Operat...|    961|
|[122729, Simulink...|    961|
|[36721, developme...|    961|
|[9342, software_d...|    961|
+--------------------+-------+
only showing top 10 rows



In [0]:
# Remove nulls from the array
# array and lit (which allows us to convert None to the correct format)
jds_non_null = jds_size.withColumn("not_null", F.array_except("jds", F.array(F.lit(None))))

In [0]:
# add size (test)
jds_non_null = jds_non_null.withColumn("num_not_null", F.size(F.col("not_null")))
jds_non_null.show(10)

+--------------------+-------+--------------------+------------+
|                 jds|num_jds|            not_null|num_not_null|
+--------------------+-------+--------------------+------------+
|[125720, HR Execu...|    961|[125720, HR Execu...|          12|
|[112708, Special ...|    961|[112708, Special ...|           4|
|[115226, consulti...|    961|[115226, consulti...|          18|
|[19805, diploma, ...|    961|[19805, diploma, ...|          11|
|[80208, Compensat...|    961|[80208, Compensat...|          11|
|[64086, Storage A...|    961|[64086, Storage A...|           2|
|[48468, HR Operat...|    961|[48468, HR Operat...|          14|
|[122729, Simulink...|    961|[122729, Simulink...|          14|
|[36721, developme...|    961|[36721, developme...|          11|
|[9342, software_d...|    961|[9342, software_d...|           7|
+--------------------+-------+--------------------+------------+
only showing top 10 rows



In [0]:
# Romove the first element (which is job_no)
jds_without_no=jds_non_null.withColumn("not_null_jds", F.expr("slice(not_null, 2, size(not_null))"))

In [0]:
# add size (test)
jds_without_no = jds_without_no.withColumn("num skills", F.size(F.col("not_null_jds")))
jds_without_no.show(10)

+--------------------+-------+--------------------+------------+--------------------+----------+
|                 jds|num_jds|            not_null|num_not_null|        not_null_jds|num skills|
+--------------------+-------+--------------------+------------+--------------------+----------+
|[125720, HR Execu...|    961|[125720, HR Execu...|          12|[HR Executive, sc...|        11|
|[112708, Special ...|    961|[112708, Special ...|           4|[Special Teacher,...|         3|
|[115226, consulti...|    961|[115226, consulti...|          18|[consulting, fres...|        17|
|[19805, diploma, ...|    961|[19805, diploma, ...|          11|[diploma, machini...|        10|
|[80208, Compensat...|    961|[80208, Compensat...|          11|[Compensation, Be...|        10|
|[64086, Storage A...|    961|[64086, Storage A...|           2|[Storage Administ...|         1|
|[48468, HR Operat...|    961|[48468, HR Operat...|          14|[HR Operations, E...|        13|
|[122729, Simulink...|    961|

In [0]:
#4 expand the skills in the remaining name array to separate rows
jds_skill = jds_without_no.withColumn ("skill", F.explode(F.col("not_null_jds")))
jds_skill.show(10)

+--------------------+-------+--------------------+------------+--------------------+----------+--------------------+
|                 jds|num_jds|            not_null|num_not_null|        not_null_jds|num skills|               skill|
+--------------------+-------+--------------------+------------+--------------------+----------+--------------------+
|[125720, HR Execu...|    961|[125720, HR Execu...|          12|[HR Executive, sc...|        11|        HR Executive|
|[125720, HR Execu...|    961|[125720, HR Execu...|          12|[HR Executive, sc...|        11|           screening|
|[125720, HR Execu...|    961|[125720, HR Execu...|          12|[HR Executive, sc...|        11|           selection|
|[125720, HR Execu...|    961|[125720, HR Execu...|          12|[HR Executive, sc...|        11|           Interview|
|[125720, HR Execu...|    961|[125720, HR Execu...|          12|[HR Executive, sc...|        11|                  HR|
|[125720, HR Execu...|    961|[125720, HR Execu...|     

In [0]:
#5 get the counts of the skills again by descending order and only show top 10
jds_skill.groupBy("skill").count().sort("count", ascending=False).show (10)

+--------------------+-----+
|               skill|count|
+--------------------+-----+
|                Java| 1911|
|          Javascript| 1770|
|               Sales| 1705|
|Business Development| 1545|
|    Web Technologies| 1313|
|Communication Skills| 1305|
|         development| 1238|
|           Marketing| 1184|
|             Finance| 1078|
|                HTML| 1067|
+--------------------+-----+
only showing top 10 rows



In [0]:
# Question 3

In [0]:
jds_without_no.groupBy("num skills").count().sort("count", ascending=False).show (5)

+----------+-----+
|num skills|count|
+----------+-----+
|        10|10477|
|         5| 3432|
|         6| 3405|
|         1| 3386|
|         7| 3345|
+----------+-----+
only showing top 5 rows



In [0]:
# question 4

In [0]:
# lower case
jds_skill_lower = jds_skill.withColumn("skill", F.lower(F.col("skill")))

In [0]:
jds_skill_lower.groupBy("skill").count().sort("count", ascending=False).show (10)

+--------------------+-----+
|               skill|count|
+--------------------+-----+
|                java| 2759|
|          javascript| 2738|
|               sales| 2680|
|business development| 2108|
|           marketing| 1809|
|                 sql| 1564|
|              jquery| 1547|
|                html| 1539|
|communication skills| 1537|
|                 bpo| 1530|
+--------------------+-----+
only showing top 10 rows



In [0]:
# question 5

In [0]:
txt_ts.show(10,truncate = False)

In [0]:
# split the txt data from the txt file show into a dataframe
ts = txt_ts.selectExpr("split(value, '\t') as ts")
ts.show(10)

+--------------------+
|                  ts|
+--------------------+
|[O*NET-SOC Code, ...|
|[11-1011.00, Adob...|
|[11-1011.00, AdSe...|
|[11-1011.00, Atla...|
|[11-1011.00, Blac...|
|[11-1011.00, Comp...|
|[11-1011.00, Data...|
|[11-1011.00, Data...|
|[11-1011.00, Emai...|
|[11-1011.00, Ente...|
+--------------------+
only showing top 10 rows



In [0]:
# extract the example column
ts_=ts.withColumn("Example",F.array(F.col("ts")[1]))
ts_.show(10)

+--------------------+--------------------+
|                  ts|             Example|
+--------------------+--------------------+
|[O*NET-SOC Code, ...|           [Example]|
|[11-1011.00, Adob...|[Adobe Systems Ad...|
|[11-1011.00, AdSe...|   [AdSense Tracker]|
|[11-1011.00, Atla...|    [Atlassian JIRA]|
|[11-1011.00, Blac...|[Blackbaud The Ra...|
|[11-1011.00, Comp...|[ComputerEase con...|
|[11-1011.00, Data...|[Database reporti...|
|[11-1011.00, Data...|           [Databox]|
|[11-1011.00, Emai...|    [Email software]|
|[11-1011.00, Ente...|[Enterprise resou...|
+--------------------+--------------------+
only showing top 10 rows



In [0]:
# transfer array to string (no)
ts_string = ts_.withColumn("Example", F.explode(F.col("Example")))
#ts_string = ts_string.withColumn("CommodityTitle", F.explode(F.col("CommodityTitle")))
ts_string.show(10)

+--------------------+--------------------+
|                  ts|             Example|
+--------------------+--------------------+
|[O*NET-SOC Code, ...|             Example|
|[11-1011.00, Adob...|Adobe Systems Ado...|
|[11-1011.00, AdSe...|     AdSense Tracker|
|[11-1011.00, Atla...|      Atlassian JIRA|
|[11-1011.00, Blac...|Blackbaud The Rai...|
|[11-1011.00, Comp...|ComputerEase cons...|
|[11-1011.00, Data...|Database reportin...|
|[11-1011.00, Data...|             Databox|
|[11-1011.00, Emai...|      Email software|
|[11-1011.00, Ente...|Enterprise resour...|
+--------------------+--------------------+
only showing top 10 rows



In [0]:
# transfer the array to string
ts_string = ts_.withColumn("Example", F.explode(F.col("Example")))
ts_string.show(10)

+--------------------+--------------------+
|                  ts|             Example|
+--------------------+--------------------+
|[O*NET-SOC Code, ...|             Example|
|[11-1011.00, Adob...|Adobe Systems Ado...|
|[11-1011.00, AdSe...|     AdSense Tracker|
|[11-1011.00, Atla...|      Atlassian JIRA|
|[11-1011.00, Blac...|Blackbaud The Rai...|
|[11-1011.00, Comp...|ComputerEase cons...|
|[11-1011.00, Data...|Database reportin...|
|[11-1011.00, Data...|             Databox|
|[11-1011.00, Emai...|      Email software|
|[11-1011.00, Ente...|Enterprise resour...|
+--------------------+--------------------+
only showing top 10 rows



In [0]:
#lower case
ts_lower = ts_string.withColumn("example", F.lower(F.col("Example")))
ts_lower.show(10)

+--------------------+--------------------+
|                  ts|             example|
+--------------------+--------------------+
|[O*NET-SOC Code, ...|             example|
|[11-1011.00, Adob...|adobe systems ado...|
|[11-1011.00, AdSe...|     adsense tracker|
|[11-1011.00, Atla...|      atlassian jira|
|[11-1011.00, Blac...|blackbaud the rai...|
|[11-1011.00, Comp...|computerease cons...|
|[11-1011.00, Data...|database reportin...|
|[11-1011.00, Data...|             databox|
|[11-1011.00, Emai...|      email software|
|[11-1011.00, Ente...|enterprise resour...|
+--------------------+--------------------+
only showing top 10 rows



In [0]:
# question 5 answer: befroe join
# create a view of "jds_sql"
jds_skill_lower.createOrReplaceTempView("jds_sql")
# count the rows
jds_before = spark.sql("SELECT COUNT(*) Before_join FROM jds_sql")
# answer (before join)
jds_before.show()

+-----------+
|Before_join|
+-----------+
|     463803|
+-----------+



In [0]:
# question 5 answer: after join
# create a view of "ts_sql"
ts_lower.createOrReplaceTempView("ts_sql")
# join the two table together and count the rows
jds_before = spark.sql("SELECT COUNT(*) After_join FROM jds_sql j JOIN ts_sql t ON j.skill=t.example")
# answer (after join)
jds_before.show()

+----------+
|After_join|
+----------+
|   1101498|
+----------+



In [0]:
## alternative answer: after join
# JDs: select "skill"
#jds_test = spark.sql("select skill from jds_sql")
# create a temp view:
#jds_test.createOrReplaceTempView("jds_test")
# O*NET: select "example"
#ts_test = spark.sql("select example from ts_sql")
# create a temp view:
#ts_test.createOrReplaceTempView("ts_test")
# join the two table together and count the rows
#join_test_after = spark.sql("SELECT COUNT(*) After_join FROM jds_test j JOIN ts_test t ON j.skill=t.example")
# answer (after join)
#join_test_after.show()

In [0]:
# Question 6

In [0]:
# add CommodityTitle column
ts_add = ts_lower.withColumn("CommodityTitle",F.array(F.col("ts")[3]))
# transfer the array to string
ts_add = ts_add.withColumn("CommodityTitle", F.explode(F.col("CommodityTitle")))

In [0]:
# create a temp view:
ts_add.createOrReplaceTempView("ts_sql")
# join the two table together only
join = spark.sql("SELECT * FROM jds_sql j JOIN ts_sql t ON j.skill=t.example")

In [0]:
# question 6 answer
# create a temp view:
join.createOrReplaceTempView("join_sql")
# count the CommodityTitle, group by count
count_test = spark.sql("SELECT CommodityTitle, COUNT(*) count FROM join_sql GROUP BY CommodityTitle ORDER BY count(*) DESC")
count_test.show(10, truncate = False)

+-------------------------------------------------+------+
|CommodityTitle                                   |count |
+-------------------------------------------------+------+
|Object or component oriented development software|324521|
|Web platform development software                |298754|
|Operating system software                        |190926|
|Development environment software                 |53013 |
|Data base management system software             |44132 |
|Analytical or scientific software                |33552 |
|Web page creation and editing software           |31682 |
|Data base user interface and query software      |29436 |
|Spreadsheet software                             |18568 |
|File versioning software                         |13846 |
+-------------------------------------------------+------+
only showing top 10 rows

